In [1]:
!nvidia-smi

Mon Dec 11 02:25:31 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 470.161.03   Driver Version: 470.161.03   CUDA Version: 11.4     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   36C    P0    29W / 250W |      0MiB / 16280MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [2]:
import pandas as pd
import numpy as np
import nltk
import string
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
import datetime

nltk.download('popular', quiet=True) # for downloading packages
import warnings

# Ignore all warnings
warnings.simplefilter("ignore")


# Setting stop words from nltk library to english
stopwords = stopwords.words('english')

# Reading the dataset
df = pd.read_csv('/kaggle/input/science-qa/train.csv')


# Tokenizing the words
def tokenizer(text):
    tokens = word_tokenize(text)
    tokens = [token.strip() for token in tokens]
    return tokens

# Cleaning the input (removing punctuations, digits, and converting to lower case)
# may not be used
def clean_input(input):
    input = input.lower()
    input = input.translate(str.maketrans('', '', string.digits))
    input = input.translate(str.maketrans('', '', string.punctuation))
    input = input.strip()
    return input

# Solving the question using cosine similarity and tfidf vectorizer 
vectorizer = TfidfVectorizer(tokenizer=tokenizer, stop_words=stopwords, lowercase=True)
matrix = vectorizer.fit_transform(tuple(df['question']))
def solve(question):
    question_vector = vectorizer.transform([question])
    cos_results = cosine_similarity(question_vector, matrix)
    index = np.argmax(cos_results, axis = None)
    return cos_results, index

# Getting the response 
def get_response(question):
    # question = clean_input(question)
    question = question.lower()

    # Getting the cosine similarity results and index of the most similar question
    similarity_results, index_res = solve(question)
    response = ''
    if similarity_results[0, index_res] < 0.6:
        response = generate_response(question)
    else:
        response =  df['answer'][index_res]

#      # Get the current date and time
#     current_time = datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S")    
#     # Write the question and response to a file
#     with open('/kaggle/working/Logs/queries.txt', 'a') as f:
#         f.write(current_time + ',' + str(similarity_results[0, index_res]) + ',' )
#         f.write( question +','+ response  +'\n')
    return response

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.24.3
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [3]:
!pip install -Uqqq pip --progress-bar off
!pip install -qqq bitsandbytes==0.39.0 --progress-bar off
!pip install -qqq torch==2.0.1 --progress-bar off
!pip install -qqq -U git+https://github.com/huggingface/transformers.git@e03a9cc --progress-bar off
!pip install -qqq -U git+https://github.com/huggingface/peft.git@42a184f --progress-bar off
!pip install -qqq -U git+https://github.com/huggingface/accelerate.git@c9fbb71 --progress-bar off
!pip install -qqq datasets==2.12.0 --progress-bar off
!pip install -qqq loralib==0.1.1 --progress-bar off
!pip install -qqq einops==0.6.1 --progress-bar off

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
kaggle-environments 1.14.3 requires transformers>=4.33.1, but you have transformers 4.30.0.dev0 which is incompatible.
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
apache-beam 2.46.0 requires dill<0.3.2,>=0.3.1.1, but you have dill 0.3.6 which is incompatible.
apache-beam 2.46.0 requires pyarrow<10.0.0,>=3.0.0, but you have pyarrow 11.0.0 which is incompatible.
pathos 0.3.1 requires dill>=0.3.7, but you have dill 0.3.6 which is incompatible.
pathos 0.3.1 requires multiprocess>=0.70.15, but you have multiprocess 0.70.14 which is incompatible.
pymc3 3.11.5 requires numpy<1.22.2,>=1.15.0, but you have numpy 1.24.3 which is incompatible.
pymc3 3.11.5 requires scipy<1.8.0,>=1.7.3, but you ha

In [4]:
import json
import os

import bitsandbytes as bnb
import pandas as pd
import torch
import torch.nn as nn
import transformers
from datasets import load_dataset
from huggingface_hub import notebook_login
from peft import (
    LoraConfig,
    PeftConfig,
    PeftModel,
    get_peft_model,
    prepare_model_for_kbit_training,
)
from transformers import (
    AutoConfig,
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
)

os.environ["CUDA_VISIBLE_DEVICES"] = "0"


===================================BUG REPORT===================================
Welcome to bitsandbytes. For bug reports, please run

python -m bitsandbytes

 and submit this information together with your error trace to: https://github.com/TimDettmers/bitsandbytes/issues
bin /opt/conda/lib/python3.10/site-packages/bitsandbytes/libbitsandbytes_cuda118_nocublaslt.so
CUDA SETUP: CUDA runtime path found: /usr/local/cuda/lib64/libcudart.so.11.0
CUDA SETUP: Highest compute capability among GPUs detected: 6.0
CUDA SETUP: Detected CUDA version 118
CUDA SETUP: Loading binary /opt/conda/lib/python3.10/site-packages/bitsandbytes/libbitsandbytes_cuda118_nocublaslt.so...


## Load Falcon Model & Tokenizer

In [5]:
MODEL_NAME = "tiiuae/falcon-7b-instruct"

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16,
)

model = AutoModelForCausalLM.from_pretrained(
    MODEL_NAME,
    device_map="auto",
    trust_remote_code=True,
    quantization_config=bnb_config,
)

tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
tokenizer.pad_token = tokenizer.eos_token

generation_config = model.generation_config
generation_config.max_new_tokens = 200
generation_config.temperature = 0.7
generation_config.top_p = 0.7
generation_config.num_return_sequences = 1
generation_config.pad_token_id = tokenizer.eos_token_id
generation_config.eos_token_id = tokenizer.eos_token_id

def generate_response(question: str) -> str:
    prompt = f"""
    {question}
    """.strip()
    encoding = tokenizer(prompt, return_tensors="pt").to("cuda:0")
    with torch.inference_mode():
        outputs = model.generate(
            input_ids=encoding.input_ids,
            attention_mask=encoding.attention_mask,
            generation_config=generation_config,
        )
    response = tokenizer.decode(outputs[0], skip_special_tokens=True)

    return response[1+len(question):].strip()

config.json:   0%|          | 0.00/1.05k [00:00<?, ?B/s]

configuration_falcon.py:   0%|          | 0.00/7.16k [00:00<?, ?B/s]

A new version of the following files was downloaded from https://huggingface.co/tiiuae/falcon-7b-instruct:
- configuration_falcon.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


modeling_falcon.py:   0%|          | 0.00/56.9k [00:00<?, ?B/s]

A new version of the following files was downloaded from https://huggingface.co/tiiuae/falcon-7b-instruct:
- modeling_falcon.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


...

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/117 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/287 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.73M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/281 [00:00<?, ?B/s]

In [6]:
prompt = "What is the least dangerous radioactive decay?"
print(get_response(prompt))

alpha decay. All radioactive decay is dangerous to living things, but alpha decay is the least dangerous.


In [7]:
prompt = "Alpha emission is a type of what?"
print(generate_response(prompt))

Alpha emission is a type of radiation that is produced by the nucleus of an atom. It is a type of electromagnetic radiation that is emitted by the nucleus of an atom when it is excited.


In [8]:
prompt = "Which category does Alpha emmision belong to?"
print(get_response(prompt))


Alpha emission is a type of radiation that is emitted by the nucleus of an atom. It is a form of electromagnetic radiation, which is a type of radiation that includes X-rays, gamma rays, and alpha particles.


In [9]:
prompt = '''What structure is made from dna and protein molecules coiled together?'''
print(get_response(prompt))

chromosome. As a cell prepares to divide, its DNA first forms one or more structures called chromosomes. A chromosome consists of DNA and protein molecules coiled into a definite shape. Chromosomes are circular in prokaryotes and rodlike in eukaryotes. You can see an example of a human chromosome in Figure below . The rest of the time, DNA looks like a tangled mass of strings. In this form, it would be very difficult to copy and divide.


In [10]:
prompt = '''"In a chemical reaction,
the amounts of reactants and products will be constant when what state is attained?"'''
print(get_response(prompt))

equilibrium. The amount of reactants and products do not have to be equal. However, after equilibrium is attained, the amounts of reactants and products will be constant.


In [11]:
prompt = '''What is the term for the force of attraction
between things that have a mass?'''
print(get_response(prompt))

gravity. Gravity has traditionally been defined as a force of attraction between things that have mass. According to this conception of gravity, anything that has mass, no matter how small, exerts gravity on other matter. Gravity can act between objects that are not even touching. In fact, gravity can act over very long distances. However, the farther two objects are from each other, the weaker is the force of gravity between them. Less massive objects also have less gravity than more massive objects.


In [12]:
prompt = '''what science says about the moon ?'''
print(get_response(prompt))

he moon is a natural satellite of Earth and is believed to have been formed from debris left over from the formation of the solar system. Scientists have been studying the moon for centuries and have discovered many interesting facts about it. Some of the most notable include the fact that the moon has no atmosphere, that it is the only natural satellite in the solar system, and that it is the only source of light on Earth during the night.
